# Only use this if you are using SSO to authenticate the notebook
from jupyterirtools import jupyterauth, athena# AWS IAM Credential Compromise Runbook 
This notebook is to be used in case of malicious activity inside your AWS Account.
We will query CloudTrail logs using Athena in order to detect and contain malicious account activity.

We are following best practices from the AWS Incident Response guide as we execute our processes:

https://docs.aws.amazon.com/whitepapers/latest/aws-security-incident-response-guide/aws-security-incident-response-guide.html

In [1]:
# Only use this if you are using SSO to authenticate the notebook
from jupyterirtools import jupyterauth, athena

AUTHTYPE: SSO
Logging in with IAM Identity Center....
Current cached SSO login is expired or invalid
Fetching credentials again


If the login window doesn't automatically open, click to [activate the session](https://device.sso.us-east-1.amazonaws.com/?user_code=XVPT-MGQS)

<IPython.core.display.Javascript object>

Waiting for login...
Waiting for login...


Credentials expire in 7 hours and 59 minutes

Login Successful


# Setup

## Load Libraries

In order to query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [2]:
pip install pyathena --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3 
boto3.setup_default_session()
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserId: {identity['UserId']}\nAccount: {identity['Account']}\nAccount: {identity['Arn']}\n")

UserId: AROA5JG7RPQDUWWUJC2OU:sso_user
Account: 913149361159
Account: arn:aws:sts::913149361159:assumed-role/AWSReservedSSO_Jupyter-IR-AdministratorAccess_df88cdf87a059be8/sso_user



In [5]:
import boto3  # the Python SDK for AWS
import pandas as pd # Pandas is a data analysis tool for Python
from pyathena import connect # Python API client for Amazon Athena
region='us-east-1' # Set region variable to us-east-1 for API commands
athena_bucket = 's3://aws-athena-query-results-us-east-1-913149361159/'  # S3 bucket that is configured to store your Athena queries

## 1. Set up Athena

### 1.1 Function to Create Tables for Queries

In [6]:
def query_results(sql):
    
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    
    columns = cursor.description
    data = cursor.fetchall()

    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]

    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    
    return df1.set_properties(**{'text-align': 'center'})

# 2.0 IAM Investigation

### 2.1 Investigating Authorization Failures
Lists failed access attempts with additional information such as the user identity ARN, time, AWS service, API call, region, source IP address, error code, and error message.

In [9]:
auth_fail ="""

select *
from "security_analysis"."cloudtrail" 
where errorCode in 
('Client.UnauthorizedOperation','Client.InvalidPermission.NotFound','Client.OperationNotPermitted','AccessDenied')
and useridentity.arn like '%iam%'
order by eventTime desc
limit 5

"""

In [12]:
df, sql = athena.run_query_sql(auth_fail)
print(sql)
df



select *
from "security_analysis"."cloudtrail" 
where errorCode in 
('Client.UnauthorizedOperation','Client.InvalidPermission.NotFound','Client.OperationNotPermitted','AccessDenied')
and useridentity.arn like '%iam%'
order by eventTime desc
limit 5




""


In [13]:
results = query_results(auth_fail)
results

,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,requestparameters,responseelements,additionaleventdata,requestid,eventid,resources,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,date_partition,region_partition,account_partition


### 2.2 Investigating the User Identities with Authorization Failures
As you know there are some failed access attempts, lets now look at the user identities with a lot of authorization failures.

In [22]:
db_name = 'security_analysis'

In [23]:
usr_auth_fail =f"""

select userIdentity.arn, count(*) as total
from "{db_name}"."cloudtrail" 
where errorCode in ('Client.UnauthorizedOperation','Client.InvalidPermission.NotFound','Client.OperationNotPermitted','AccessDenied')
and useridentity.arn like '%iam%'
group by userIdentity.arn
order by total desc

"""

In [24]:
results = query_results(usr_auth_fail)
results

,arn,total


### 2.3 Investigating an IAM User
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what the entity has been attemping to do.

In [25]:
iam_usr =f"""

select *
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
limit 5

"""

In [26]:
results = query_results(iam_usr)
results

,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,requestparameters,responseelements,additionaleventdata,requestid,eventid,resources,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,date_partition,region_partition,account_partition


### 2.4 Investigation the AWS services Used by the Suspicious Identity
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what services the suspicious identity has been attempting to use.

In [27]:
services_used =f"""

select eventSource, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventSource
order by total desc

"""

In [28]:
results = query_results(services_used)
results

,eventSource,total


### 2.5 Investigation the Action Attempted by the Suspicious Identity
Now that you have identified the potential IAM entity which has been compromised, you should do further investigations to identify what action the suspicious identity has been attempting to take.

In [29]:
act_attempt = f'''
select eventName, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventName
order by total desc
'''

In [30]:
results = query_results(act_attempt)
results

,eventName,total


### 2.6 Bringing it all together
Bring together the previous queries to create a single query showing the event name, AWS service, and AWS region where requests were being made to by the compromised IAM entity.

In [31]:
service_apicalls_time = f'''

select eventTime, eventSource, eventName, awsregion, count(*) AS total 
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by eventName, eventSource, eventTime, awsregion
order by eventTime DESC

'''

In [32]:
results = query_results(service_apicalls_time)
results

,eventTime,eventSource,eventName,awsregion,total


### Title of the query
Insert description about the purpose and expected output of the query

In [ ]:
query = '''

'''

In [ ]:
results = query_results(query)
results

## 3. IAM Containment
Now that you have identified the potential IAM entity which has been compromised you need to perform containment activities. The first of these will be to find out what the Access Key ID is being used by the account.


### 3.1 Identify Access Key ID

In [ ]:
sql = f'''

select useridentity.accesskeyid, count(*) as total
from "{db_name}"."cloudtrail" 
where userIdentity.arn='insert ARN of IAM Credential from query in 2.2'
group by useridentity.accesskeyid
order by total desc

'''

In [ ]:
results = query_results(sql)
results

### 3.2 Deactivate Access Key

In [ ]:
access_key_to_deactivate='ENTER ACCESS KEY ID HERE'
username='ENTER USERNAME HERE'
iam = boto3.resource('iam', region_name=region)
access_key = iam.AccessKey(username,access_key_to_deactivate)
response_status = access_key.deactivate()
status_code = response_status['ResponseMetadata']['HTTPStatusCode']
if status_code == 200:
    print('Key Disabled Successfully')
else:
    print('Key deactivation failed')

### 3.3 Attach Deny All Actions Policy

In [ ]:
username='ENTER USERNAME HERE'
iam = boto3.client('iam', region_name=region)
response = iam.put_user_policy(UserName=username,PolicyName='Block',PolicyDocument='{"Version":"2012-10-17","Statement":{"Effect":"Deny","Action":"*","Resource":"*"}}')
status_code = response['ResponseMetadata']['HTTPStatusCode']
if status_code == 200:
    print('Policy attached successfully')
else:
    print('Policy attachment failed')
